# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123783e+02     1.485722e+00
 * time: 0.8032829761505127
     1     1.128298e+01     8.361269e-01
 * time: 2.247912883758545
     2    -1.278105e+01     9.886060e-01
 * time: 2.964488983154297
     3    -3.469771e+01     7.561409e-01
 * time: 3.989982843399048
     4    -4.860638e+01     5.258424e-01
 * time: 5.037807941436768
     5    -5.731292e+01     1.916179e-01
 * time: 6.040586948394775
     6    -5.993125e+01     1.149274e-01
 * time: 6.718851804733276
     7    -6.088695e+01     5.645686e-02
 * time: 7.40641188621521
     8    -6.129148e+01     8.688166e-02
 * time: 8.109572887420654
     9    -6.160379e+01     2.891149e-02
 * time: 8.783873796463013
    10    -6.181233e+01     2.606852e-02
 * time: 9.459289789199829
    11    -6.196458e+01     2.076499e-02
 * time: 10.158759832382202
    12    -6.204160e+01     1.525077e-02
 * time: 10.860054969787598
    13    -6.210870e+01     1.468410e-02
 * time: 11.547191858291626
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671086
    AtomicLocal         -18.8557739
    AtomicNonlocal      14.8522669
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485406 
    Xc                  -19.3336834

    total               -62.261666458289
